In [1]:
import pandas as pd
import os
import tensorflow as tf

import scipy
from scipy.io import loadmat
import re

import string
import imageio
import numpy as np
import matplotlib.pyplot as plt
import logging, scipy, multiprocessing
import matplotlib.image as img
import random
import time
import nltk
import cv2
import warnings
import tensorflow as tf
import tensorlayer as tl
import keras
import tensorflow as tf 
from tensorlayer.layers import *
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# Load Data

In [2]:
train_hr_img_list = sorted(tl.files.load_file_list(path=".\image_2", regx='.*.png', printable=False))
train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=".\image_2", n_threads=32)

[TL] read 32 from .\image_2
[TL] read 64 from .\image_2
[TL] read 96 from .\image_2
[TL] read 128 from .\image_2
[TL] read 160 from .\image_2
[TL] read 192 from .\image_2
[TL] read 224 from .\image_2
[TL] read 256 from .\image_2
[TL] read 288 from .\image_2
[TL] read 320 from .\image_2
[TL] read 352 from .\image_2
[TL] read 384 from .\image_2
[TL] read 416 from .\image_2
[TL] read 448 from .\image_2
[TL] read 480 from .\image_2
[TL] read 512 from .\image_2
[TL] read 544 from .\image_2
[TL] read 576 from .\image_2
[TL] read 608 from .\image_2
[TL] read 640 from .\image_2
[TL] read 672 from .\image_2
[TL] read 704 from .\image_2
[TL] read 736 from .\image_2
[TL] read 768 from .\image_2
[TL] read 800 from .\image_2
[TL] read 832 from .\image_2
[TL] read 864 from .\image_2
[TL] read 896 from .\image_2
[TL] read 928 from .\image_2
[TL] read 960 from .\image_2
[TL] read 992 from .\image_2
[TL] read 1024 from .\image_2
[TL] read 1056 from .\image_2
[TL] read 1088 from .\image_2
[TL] read 1120

In [34]:
def generator_train():
        for img in train_hr_imgs:
            yield img
def _map_fn_train(img):
        hr_patch = tf.image.random_crop(img, [256, 256, 3])
        hr_patch = hr_patch / (255. / 2.)
        hr_patch = hr_patch - 1.
        hr_patch = tf.image.random_flip_left_right(hr_patch)
        lr_patch = tf.image.resize_images(hr_patch, (64, 64), method=2)
        #lr_patch = tf.image.resize(hr_patch, size=[64, 64])
        return lr_patch, hr_patch
def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image
    return img

In [35]:
def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def save_images(images, size, image_path):
    return imsave(images, size, image_path)

In [36]:
train_ds = tf.data.Dataset.from_generator(generator_train, output_types=(tf.float32))
train_ds = train_ds.map(_map_fn_train, num_parallel_calls=multiprocessing.cpu_count())
train_ds = train_ds.repeat()
train_ds = train_ds.shuffle(3*9)
#train_ds = train_ds.prefetch(buffer_size=256)
train_ds = train_ds.batch(9)

In [37]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

In [38]:
iterator_oneshot = train_ds.make_one_shot_iterator()
next_element_oneshot = iterator_oneshot.get_next()
a, b = sess.run(next_element_oneshot)
print(a.shape,b.shape)

(9, 64, 64, 3) (9, 256, 256, 3)


# Model

In [39]:
def deconv2d(input_, output_shape,
             k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
             name="deconv2d"):
    with tf.variable_scope(name):
        
        w = tf.get_variable('w', [k_h, k_h, output_shape[-1], input_.get_shape()[-1]],
                            initializer=tf.random_normal_initializer(stddev=stddev))


        deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,strides=[1, d_h, d_w, 1])



        biases = tf.get_variable('biases', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())
        
        return deconv

In [40]:
class Generator:
    def __init__(self, noise_z, training_phase, hparas, reuse):
        self.z = noise_z
        self.train = training_phase
        self.hparas = hparas
        self.reuse = reuse
        self._build_model()
    def _build_model(self):
        """ Generator in Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network
        feature maps (n) and stride (s) feature maps (n) and stride (s)
        """
        relu = tf.nn.relu
        tanh = tf.nn.tanh
        conv2d = tf.layers.conv2d
        bn = tf.layers.batch_normalization
        linear = tf.layers.dense
        with tf.variable_scope("generator", reuse=self.reuse) as vs:
            n = relu(conv2d(self.z, 64, kernel_size=3, strides=1, padding='same'))
            temp = n
            # B residual blocks
            for i in range(16):
                nn = conv2d(n, 64, kernel_size=3, strides=1, padding='same')
                nn = relu(bn(nn,training = self.train ))
                nn = conv2d(nn, 64, kernel_size=3, strides=1, padding='same')
                nn = bn(nn,training = self.train )
                nn = tf.add(n,nn)
            n = conv2d(n, 64, kernel_size=3, strides=1, padding='same')
            n = bn(nn,training = self.train )
            n = tf.add(n,temp)
            # B residual blacks end
            n = conv2d(n, 256, kernel_size=3, strides=1, padding='same')
            n = relu(deconv2d(n, [9, 128, 128 , 256], name='g_h1'))
            n = conv2d(n, 256, kernel_size=3, strides=1, padding='same')
            n = relu(deconv2d(n, [9, 256, 256 , 256], name='g_h2'))
            n = tanh(conv2d(n, 3, kernel_size=3, strides=1, padding='same'))
            self.outputs = n

In [41]:
# resnet structure
class Discriminator:
    def __init__(self, image, training_phase, hparas, reuse):
        self.image = image
        self.train = training_phase
        self.hparas = hparas
        self.reuse = reuse
        self._build_model()
    
    def _build_model(self): 
        lrelu = tf.nn.leaky_relu
        conv2d = tf.layers.conv2d
        bn = tf.layers.batch_normalization
        dense = tf.layers.dense
        df_dim = 64
        with tf.variable_scope('discriminator', reuse=self.reuse):
            n = lrelu(conv2d(self.image, df_dim, kernel_size=4, strides=2, padding='same'))
            n = lrelu(bn(conv2d(n, df_dim*2 , kernel_size=4, strides=2, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*4 , kernel_size=4, strides=2, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*8 , kernel_size=4, strides=2, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*16 , kernel_size=4, strides=2, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*32 , kernel_size=4, strides=2, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*16 , kernel_size=1, strides=1, padding='same'),training = self.train))
            nn = bn(conv2d(n, df_dim*8 , kernel_size=1, strides=1, padding='same'),training = self.train)
            n = lrelu(bn(conv2d(nn, df_dim*2 , kernel_size=1, strides=1, padding='same'),training = self.train))
            n = lrelu(bn(conv2d(n, df_dim*2 , kernel_size=3, strides=1, padding='same'),training = self.train))
            n = bn(conv2d(n, df_dim*8 , kernel_size=3, strides=1, padding='same'),training = self.train)
            n = tf.add(n,nn)
            n = tf.layers.Flatten()(n)
            d_net = dense(n,1)
            self.logits = d_net
            #self.discriminator_net = d_net
            self.outputs =  d_net

In [42]:
def get_hparas():
    hparas = {
        'BATCH_SIZE' : 9,
        'LR' : 1e-4,
        'BETA' : 0.9, # AdamOptimizer parameter
        'N_EPOCH' : 100,
        'N_SAMPLE' : 7480
    }
    return hparas

In [67]:
class GAN:
    def __init__(self, hparas, training_phase, ckpt_path, inference_path, test_img = None,recover=None):
        self.hparas = hparas
        self.train = training_phase
        self.ckpt_path = ckpt_path
        self.sample_path = './samples'
        self.inference_path = './inference'
        
        self._get_session() # get session
        self._get_train_data_iter() # initialize and get data iterator
        self._input_layer() # define input placeholder
        self._get_inference() # build generator and discriminator
        self._get_loss() # define gan loss
        self._get_var_with_name() # get variables for each part of model
        self._optimize() # define optimizer
        self._init_vars()
        self._get_saver()
        self.test_LRbatch = test_img
        if recover is not None:
            self._load_checkpoint(recover)
            #self.test_LRbatch = test_img
            
            
        #print (self.generator  )  
        
    def _get_train_data_iter(self):
        if self.train: # training data iteratot
            iterator_oneshot = train_ds.make_one_shot_iterator()
            self.next_element_oneshot = iterator_oneshot.get_next()
           
            
    def _input_layer(self):
        if self.train:
            self.real_image = tf.placeholder(tf.float32,[None, 256,256, 3],name='real_image')
            self.z_noise = tf.placeholder(tf.float32, [None, 64,64,3], name='z_noise')
        else:
            
            self.z_noise = tf.placeholder(tf.float32, [None, 64,64,3], name='z_noise')
    
    def _get_inference(self):
        if self.train:
            
            # generating image
            generator = Generator(self.z_noise, training_phase=True,hparas=self.hparas, reuse=False)
            self.generator = generator
            # discriminize
            # fake image
            fake_discriminator = Discriminator(generator.outputs,training_phase=True, hparas=self.hparas, reuse=False)
            self.fake_discriminator = fake_discriminator
            # real image
            real_discriminator = Discriminator(self.real_image, training_phase=True,hparas=self.hparas, reuse=True)
            self.real_discriminator = real_discriminator
            
        else: # inference mode
            self.generate_image_net = Generator(self.z_noise, training_phase=False,hparas=self.hparas, reuse=False)
            
    def _get_loss(self):
        if self.train:
            d_loss1 = tl.cost.sigmoid_cross_entropy(self.real_discriminator.logits, tf.ones_like(self.real_discriminator.logits), name='d1')
            d_loss2 = tl.cost.sigmoid_cross_entropy(self.fake_discriminator.logits, tf.zeros_like(self.fake_discriminator.logits), name='d2')
            self.d_loss = d_loss1 + d_loss2
            mse_loss = tl.cost.mean_squared_error(self.generator.outputs, self.real_image, is_mean=True)
            self.initmse_loss = mse_loss
            g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(self.fake_discriminator.logits, tf.ones_like(self.fake_discriminator.logits), name='g')
            self.g_loss = mse_loss + g_gan_loss 
    
    def _optimize(self):
        if self.train:
            with tf.variable_scope('learning_rate'):
                self.lr_var = tf.Variable(self.hparas['LR'], trainable=False)

            discriminator_optimizer = tf.train.AdamOptimizer(self.lr_var, beta1=self.hparas['BETA'])
            generator_optimizer = tf.train.AdamOptimizer(self.lr_var, beta1=self.hparas['BETA'])
            g_optim_init_optimizer = tf.train.AdamOptimizer(self.lr_var, beta1=self.hparas['BETA'])
            
            self.d_optim = discriminator_optimizer.minimize(self.d_loss, var_list=self.discrim_vars)
            self.g_optim = generator_optimizer.minimize(self.g_loss, var_list=self.generator_vars)
            self.g_optim_init = g_optim_init_optimizer.minimize(self.initmse_loss, var_list=self.generator_vars)
        
    def training(self):
        
        for _epoch in range(self.hparas['N_EPOCH']):
            start_time = time.time()
            n_batch_epoch = int(self.hparas['N_SAMPLE']/self.hparas['BATCH_SIZE'])
            if _epoch < 5:
                for _step in range(n_batch_epoch):
                    step_time = time.time()
                    lr_batch, hr_batch = self.sess.run(self.next_element_oneshot)
                    self.lr_batch = lr_batch
                    self.hr_batch = hr_batch
                    self.generator_error, _ = self.sess.run([self.initmse_loss, self.g_optim_init],
                                                           feed_dict={self.z_noise:self.lr_batch ,
                                                                      self.real_image:self.hr_batch})
                    if _step%50==0:
                        print("Epoch: [%2d/%2d] [%4d/%4d] time: %4.4fs, g_loss: %.3f" \
                                % (_epoch, self.hparas['N_EPOCH'], _step, n_batch_epoch, time.time() - step_time,
                                    self.generator_error))
            else:
                for _step in range(n_batch_epoch):
                    step_time = time.time()
                    lr_batch, hr_batch = self.sess.run(self.next_element_oneshot)
                    self.lr_batch = lr_batch
                    self.hr_batch = hr_batch
                    # update discriminator
                    self.discriminator_error, _ = self.sess.run([self.d_loss, self.d_optim],
                                                                   feed_dict={
                                                                        self.real_image:self.hr_batch,
                                                                        self.z_noise:self.lr_batch})

                    # update generate
                    self.generator_error, _ = self.sess.run([self.g_loss, self.g_optim],
                                                           feed_dict={self.z_noise:self.lr_batch ,
                                                                      self.real_image:self.hr_batch})


                    if _step%50==0:
                        print("Epoch: [%2d/%2d] [%4d/%4d] time: %4.4fs, d_loss: %.3f, g_loss: %.3f" \
                                % (_epoch, self.hparas['N_EPOCH'], _step, n_batch_epoch, time.time() - step_time,
                                   self.discriminator_error, self.generator_error))
                    
                self._save_checkpoint(_epoch)
            self._sample_visiualize(_epoch)
            #self._sample_visiualize(_epoch)
            
    def inference(self):
        img_gen = self.sess.run(self.generate_image_net.outputs, feed_dict={self.z_noise : self.test_LRbatch})
        save_images(img_gen, [3, 3], './test_img/SRcorp.png')
        print('-----success save SRcrop--------')
        
    def _init_vars(self):
        self.sess.run(tf.global_variables_initializer())
    
    def _get_session(self):
        self.sess = tf.Session()
    
    def _get_saver(self):
        if self.train:
            self.g_saver = tf.train.Saver(var_list=self.generator_vars)
            self.d_saver = tf.train.Saver(var_list=self.discrim_vars)
        else:
            self.g_saver = tf.train.Saver(var_list=self.generator_vars)
            
    def _sample_visiualize(self, epoch):
        ni = int(np.ceil(np.sqrt(self.hparas['BATCH_SIZE'])))
    
            
        img_gen = self.sess.run(self.generator.outputs,feed_dict={self.z_noise : self.test_LRbatch})
        #img_hr = self.hr_batch
        #save_images(img_hr, [ni, ni], './SR_training/HRtrain_{:02d}.png'.format(epoch))
        save_images(img_gen, [ni, ni], './SR_training/Gentrain_{:02d}.png'.format(epoch))
        
    def _get_var_with_name(self):
        t_vars = tf.trainable_variables()
        self.generator_vars = [var for var in t_vars if 'generator' in var.name]
        self.discrim_vars = [var for var in t_vars if 'discriminator' in var.name]
    
    def _load_checkpoint(self, recover):
        if self.train:
            self.g_saver.restore(self.sess, self.ckpt_path+'g_model_'+str(recover)+'.ckpt')
            self.d_saver.restore(self.sess, self.ckpt_path+'d_model_'+str(recover)+'.ckpt')
        else:
            self.g_saver.restore(self.sess, self.ckpt_path+'g_model_'+str(recover)+'.ckpt')
        print('-----success restored checkpoint--------')
    
    def _save_checkpoint(self, epoch):
        self.g_saver.save(self.sess, self.ckpt_path+'g_model_'+str(epoch)+'.ckpt')
        self.d_saver.save(self.sess, self.ckpt_path+'d_model_'+str(epoch)+'.ckpt')
        print('-----success saved checkpoint--------')

# Training

In [69]:
from PIL import Image
crop_img = Image.open('./test_img/output060.png')
#plt.imshow(crop_img)
testLR_img = []
width, height = crop_img.size
start_pos = start_x, start_y = (0, 0)
cropped_image_size = w, h = (64, 64)

for row_i in range(0, width, w):
    for col_i in range(0, height, h):
        lr = crop_img.crop((col_i, row_i, col_i + w, row_i + h))
        pic = np.array(lr)
        testLR_img.append(pic)
LRimgs = np.array(testLR_img)
print(LRimgs.shape)
LRimgs = LRimgs / (255. / 2.)
LRimgs = LRimgs - 1.
#print(LRimgs)

(9, 64, 64, 3)


In [70]:
tf.reset_default_graph()
checkpoint_path = './checkpoint/'
inference_path = './inference'
gan = GAN(get_hparas(), training_phase=True, ckpt_path=checkpoint_path, inference_path=inference_path,test_img = LRimgs)
gan.training()

Epoch: [ 0/100] [   0/ 831] time: 7.0601s, g_loss: 0.544
Epoch: [ 0/100] [  50/ 831] time: 1.8854s, g_loss: 0.059
Epoch: [ 0/100] [ 100/ 831] time: 1.1094s, g_loss: 0.039
Epoch: [ 0/100] [ 150/ 831] time: 0.7228s, g_loss: 0.032
Epoch: [ 0/100] [ 200/ 831] time: 0.7428s, g_loss: 0.043
Epoch: [ 0/100] [ 250/ 831] time: 1.1172s, g_loss: 0.029
Epoch: [ 0/100] [ 300/ 831] time: 0.7161s, g_loss: 0.030
Epoch: [ 0/100] [ 350/ 831] time: 0.6858s, g_loss: 0.026
Epoch: [ 0/100] [ 400/ 831] time: 0.6855s, g_loss: 0.037
Epoch: [ 0/100] [ 450/ 831] time: 0.6948s, g_loss: 0.032
Epoch: [ 0/100] [ 500/ 831] time: 0.6887s, g_loss: 0.037
Epoch: [ 0/100] [ 550/ 831] time: 0.6908s, g_loss: 0.032
Epoch: [ 0/100] [ 600/ 831] time: 0.7472s, g_loss: 0.026
Epoch: [ 0/100] [ 650/ 831] time: 1.9152s, g_loss: 0.027
Epoch: [ 0/100] [ 700/ 831] time: 1.0252s, g_loss: 0.022
Epoch: [ 0/100] [ 750/ 831] time: 0.8527s, g_loss: 0.025
Epoch: [ 0/100] [ 800/ 831] time: 1.5358s, g_loss: 0.028
Epoch: [ 1/100] [   0/ 831] tim

Epoch: [14/100] [ 100/ 831] time: 1.4728s, d_loss: 0.464, g_loss: 0.031
Epoch: [14/100] [ 150/ 831] time: 1.2463s, d_loss: 0.852, g_loss: 0.034
Epoch: [14/100] [ 200/ 831] time: 1.3043s, d_loss: 0.708, g_loss: 0.024
Epoch: [14/100] [ 250/ 831] time: 1.3244s, d_loss: 2.564, g_loss: 0.032
Epoch: [14/100] [ 300/ 831] time: 1.4027s, d_loss: 0.887, g_loss: 0.023
Epoch: [14/100] [ 350/ 831] time: 1.2870s, d_loss: 1.158, g_loss: 0.027
Epoch: [14/100] [ 400/ 831] time: 1.2480s, d_loss: 1.616, g_loss: 0.025
Epoch: [14/100] [ 450/ 831] time: 1.2398s, d_loss: 0.031, g_loss: 0.032
Epoch: [14/100] [ 500/ 831] time: 2.4613s, d_loss: 1.348, g_loss: 0.038
Epoch: [14/100] [ 550/ 831] time: 1.2638s, d_loss: 1.176, g_loss: 0.036
Epoch: [14/100] [ 600/ 831] time: 1.3277s, d_loss: 0.127, g_loss: 0.036
Epoch: [14/100] [ 650/ 831] time: 1.3271s, d_loss: 0.103, g_loss: 0.026
Epoch: [14/100] [ 700/ 831] time: 1.3078s, d_loss: 0.103, g_loss: 0.036
Epoch: [14/100] [ 750/ 831] time: 1.8247s, d_loss: 1.215, g_loss

Epoch: [27/100] [  50/ 831] time: 1.3044s, d_loss: 3.083, g_loss: 0.024
Epoch: [27/100] [ 100/ 831] time: 1.2466s, d_loss: 0.124, g_loss: 0.029
Epoch: [27/100] [ 150/ 831] time: 1.2595s, d_loss: 0.176, g_loss: 0.033
Epoch: [27/100] [ 200/ 831] time: 1.2827s, d_loss: 0.240, g_loss: 0.022
Epoch: [27/100] [ 250/ 831] time: 1.2594s, d_loss: 0.200, g_loss: 0.031
Epoch: [27/100] [ 300/ 831] time: 1.2596s, d_loss: 0.419, g_loss: 0.030
Epoch: [27/100] [ 350/ 831] time: 1.2417s, d_loss: 0.078, g_loss: 0.035
Epoch: [27/100] [ 400/ 831] time: 1.2419s, d_loss: 2.889, g_loss: 0.026
Epoch: [27/100] [ 450/ 831] time: 1.3036s, d_loss: 0.633, g_loss: 0.033
Epoch: [27/100] [ 500/ 831] time: 1.2308s, d_loss: 0.438, g_loss: 0.028
Epoch: [27/100] [ 550/ 831] time: 1.2538s, d_loss: 0.558, g_loss: 0.023
Epoch: [27/100] [ 600/ 831] time: 1.2580s, d_loss: 0.299, g_loss: 0.023
Epoch: [27/100] [ 650/ 831] time: 1.7621s, d_loss: 1.038, g_loss: 0.031
Epoch: [27/100] [ 700/ 831] time: 1.2491s, d_loss: 0.341, g_loss

Epoch: [40/100] [   0/ 831] time: 3.1891s, d_loss: 0.429, g_loss: 0.016
Epoch: [40/100] [  50/ 831] time: 1.3296s, d_loss: 0.745, g_loss: 0.025
Epoch: [40/100] [ 100/ 831] time: 1.2936s, d_loss: 0.105, g_loss: 0.033
Epoch: [40/100] [ 150/ 831] time: 1.2846s, d_loss: 2.779, g_loss: 0.034
Epoch: [40/100] [ 200/ 831] time: 1.3916s, d_loss: 0.616, g_loss: 0.032
Epoch: [40/100] [ 250/ 831] time: 1.2606s, d_loss: 0.222, g_loss: 0.030
Epoch: [40/100] [ 300/ 831] time: 8.8296s, d_loss: 2.998, g_loss: 0.026
Epoch: [40/100] [ 350/ 831] time: 1.2509s, d_loss: 0.067, g_loss: 0.034
Epoch: [40/100] [ 400/ 831] time: 1.2211s, d_loss: 0.643, g_loss: 0.034
Epoch: [40/100] [ 450/ 831] time: 1.2656s, d_loss: 0.148, g_loss: 0.029
Epoch: [40/100] [ 500/ 831] time: 1.2344s, d_loss: 0.418, g_loss: 0.023
Epoch: [40/100] [ 550/ 831] time: 1.2500s, d_loss: 0.047, g_loss: 0.039
Epoch: [40/100] [ 600/ 831] time: 1.2187s, d_loss: 0.201, g_loss: 0.037
Epoch: [40/100] [ 650/ 831] time: 1.3125s, d_loss: 0.534, g_loss

Epoch: [52/100] [ 800/ 831] time: 1.2796s, d_loss: 0.054, g_loss: 0.030
-----success saved checkpoint--------
Epoch: [53/100] [   0/ 831] time: 1.2586s, d_loss: 2.227, g_loss: 0.030
Epoch: [53/100] [  50/ 831] time: 1.3926s, d_loss: 0.131, g_loss: 0.028
Epoch: [53/100] [ 100/ 831] time: 1.2516s, d_loss: 0.087, g_loss: 0.031
Epoch: [53/100] [ 150/ 831] time: 1.3796s, d_loss: 0.080, g_loss: 0.034
Epoch: [53/100] [ 200/ 831] time: 1.2646s, d_loss: 3.822, g_loss: 0.019
Epoch: [53/100] [ 250/ 831] time: 1.3106s, d_loss: 0.884, g_loss: 0.039
Epoch: [53/100] [ 300/ 831] time: 1.2616s, d_loss: 0.345, g_loss: 0.021
Epoch: [53/100] [ 350/ 831] time: 1.2976s, d_loss: 0.243, g_loss: 0.025
Epoch: [53/100] [ 400/ 831] time: 1.2656s, d_loss: 0.573, g_loss: 0.033
Epoch: [53/100] [ 450/ 831] time: 1.3066s, d_loss: 1.437, g_loss: 0.031
Epoch: [53/100] [ 500/ 831] time: 1.2976s, d_loss: 0.144, g_loss: 0.027
Epoch: [53/100] [ 550/ 831] time: 1.2176s, d_loss: 0.605, g_loss: 0.033
Epoch: [53/100] [ 600/ 831

Epoch: [65/100] [ 750/ 831] time: 1.4476s, d_loss: 0.035, g_loss: 0.035
Epoch: [65/100] [ 800/ 831] time: 1.3076s, d_loss: 0.390, g_loss: 0.028
-----success saved checkpoint--------
Epoch: [66/100] [   0/ 831] time: 1.2726s, d_loss: 1.111, g_loss: 0.030
Epoch: [66/100] [  50/ 831] time: 1.3006s, d_loss: 0.070, g_loss: 0.035
Epoch: [66/100] [ 100/ 831] time: 1.4186s, d_loss: 0.245, g_loss: 0.037
Epoch: [66/100] [ 150/ 831] time: 1.2496s, d_loss: 0.035, g_loss: 0.028
Epoch: [66/100] [ 200/ 831] time: 4.1778s, d_loss: 0.009, g_loss: 0.046
Epoch: [66/100] [ 250/ 831] time: 1.4226s, d_loss: 0.442, g_loss: 0.024
Epoch: [66/100] [ 300/ 831] time: 1.2566s, d_loss: 0.629, g_loss: 0.025
Epoch: [66/100] [ 350/ 831] time: 1.2496s, d_loss: 0.103, g_loss: 0.023
Epoch: [66/100] [ 400/ 831] time: 1.3646s, d_loss: 0.018, g_loss: 0.031
Epoch: [66/100] [ 450/ 831] time: 1.2476s, d_loss: 0.026, g_loss: 0.037
Epoch: [66/100] [ 500/ 831] time: 1.2406s, d_loss: 0.037, g_loss: 0.044
Epoch: [66/100] [ 550/ 831

Epoch: [78/100] [ 700/ 831] time: 1.2896s, d_loss: 0.002, g_loss: 0.040
Epoch: [78/100] [ 750/ 831] time: 1.2456s, d_loss: 0.000, g_loss: 0.037
Epoch: [78/100] [ 800/ 831] time: 1.3876s, d_loss: 0.000, g_loss: 0.050
-----success saved checkpoint--------
Epoch: [79/100] [   0/ 831] time: 2.1024s, d_loss: 0.674, g_loss: 0.031
Epoch: [79/100] [  50/ 831] time: 1.2826s, d_loss: 0.001, g_loss: 0.035
Epoch: [79/100] [ 100/ 831] time: 1.3126s, d_loss: 0.004, g_loss: 0.025
Epoch: [79/100] [ 150/ 831] time: 1.3296s, d_loss: 0.005, g_loss: 0.030
Epoch: [79/100] [ 200/ 831] time: 4.1938s, d_loss: 0.000, g_loss: 0.036
Epoch: [79/100] [ 250/ 831] time: 2.9571s, d_loss: 0.000, g_loss: 0.039
Epoch: [79/100] [ 300/ 831] time: 1.2626s, d_loss: 0.000, g_loss: 0.055
Epoch: [79/100] [ 350/ 831] time: 1.2486s, d_loss: 0.619, g_loss: 0.031
Epoch: [79/100] [ 400/ 831] time: 1.2446s, d_loss: 0.000, g_loss: 0.041
Epoch: [79/100] [ 450/ 831] time: 1.4296s, d_loss: 0.011, g_loss: 0.037
Epoch: [79/100] [ 500/ 831

Epoch: [91/100] [ 650/ 831] time: 1.3086s, d_loss: 0.036, g_loss: 0.027
Epoch: [91/100] [ 700/ 831] time: 1.2856s, d_loss: 0.003, g_loss: 0.040
Epoch: [91/100] [ 750/ 831] time: 1.3296s, d_loss: 0.002, g_loss: 0.050
Epoch: [91/100] [ 800/ 831] time: 1.2456s, d_loss: 0.000, g_loss: 0.037
-----success saved checkpoint--------
Epoch: [92/100] [   0/ 831] time: 1.3206s, d_loss: 0.000, g_loss: 0.039
Epoch: [92/100] [  50/ 831] time: 1.3206s, d_loss: 0.006, g_loss: 0.031
Epoch: [92/100] [ 100/ 831] time: 1.3736s, d_loss: 0.004, g_loss: 0.040
Epoch: [92/100] [ 150/ 831] time: 1.2766s, d_loss: 0.000, g_loss: 0.031
Epoch: [92/100] [ 200/ 831] time: 2.5522s, d_loss: 0.000, g_loss: 0.041
Epoch: [92/100] [ 250/ 831] time: 1.2966s, d_loss: 0.002, g_loss: 0.041
Epoch: [92/100] [ 300/ 831] time: 1.3196s, d_loss: 0.002, g_loss: 0.039
Epoch: [92/100] [ 350/ 831] time: 1.2836s, d_loss: 0.011, g_loss: 0.037
Epoch: [92/100] [ 400/ 831] time: 2.8542s, d_loss: 0.001, g_loss: 0.044
Epoch: [92/100] [ 450/ 831

# Inference

In [177]:
checkpoint_path = './checkpoint/'
inference_path = './inference'
tf.reset_default_graph()
gan = GAN(get_hparas(), training_phase=False, ckpt_path=checkpoint_path, inference_path=inference_path,test_img = LRimgs ,recover=42)
img = gan.inference()

INFO:tensorflow:Restoring parameters from ./checkpoint/g_model_42.ckpt


INFO:tensorflow:Restoring parameters from ./checkpoint/g_model_42.ckpt


-----success restored checkpoint--------
-----success save SRcrop--------
